In [12]:
import pandas as pd
import streamlit as st
import snowflake.connector
from dotenv import load_dotenv
import os

In [13]:
load_dotenv()
rapidapi_key = os.getenv('RAPIDAPI_KEY')
rapidapi_host = "linkedin-job-search-api.p.rapidapi.com"
snowflake_password = os.getenv('SNOWFLAKE_PASSWORD')

## Snowflake Connection

In [14]:
# Connect to Snowflake
#Establish a connection to Snowflake

def connect_to_snowflake():
    try:

        conn = snowflake.connector.connect(
            user="NIKKILW2025",
            password=snowflake_password,
            account="gbszkwp-by30611",
            warehouse="SNOWFLAKE_LEARNING_WH",
            database="linkedin_db",
            schema="linkedin_raw"
        )
        print("Connection to Snowflake established successfully.")
        return conn
    except Exception as e:
        print(f"Error connecting to Snowflake: {e}")
        return None

conn = connect_to_snowflake()

Connection to Snowflake established successfully.


In [ ]:
def query_job_total(conn):
    query = """
        SELECT JOB_CATEGORY as Title,
        COUNT(DISTINCT ID) as Total_Jobs
        FROM LINKEDIN_JOB_API_CLEANED_DATA
        WHERE
        lower(title) LIKE '%data engineer%'
        or lower(title) LIKE '%data analyst%'
        or lower(title) LIKE '%data scientist%'
        GROUP BY JOB_CATEGORY
        ORDER BY Title ASC
    """
    df_job_total = pd.read_sql(query, conn)
    return df_job_total

df_job_total = query_job_total(conn)
df_job_total


In [ ]:
# def job_data_summary(df_job_total):
#     """
#     3 Summary Numbers of Total Jobs by Title
#     """
#     col1, col2, col3 = st.columns(3)

#     col1.metric(label='Data Analyst', value=int(df_job_total.iloc[0,1]))
#     col2.metric(label='Data Engineer', value=int(df_job_total.iloc[1,1]))
#     col3.metric(label='Data Scientist', value=int(df_job_total.iloc[2,1]))


In [49]:
##Get Daily jobs trend by title
def query_daily_job_data(conn):
    query = """
        SELECT
        DATE,
        JOB_CATEGORY as Title,
        SUM(TOTAL_JOBS) as Total_Jobs
        FROM MART_TOTAL_JOBS_DAILY
        WHERE
        lower(title) LIKE '%data engineer%'
        or lower(title) LIKE '%data analyst%'
        or lower(title) LIKE '%data scientist%'
        GROUP BY Title, DATE
        ORDER BY Title, DATE ASC
    """
    df_daily_jobs = pd.read_sql(query, conn)
    return df_daily_jobs

df_daily_jobs = query_daily_job_data(conn)
df_daily_jobs.head()

/tmp/ipykernel_15211/4181880399.py:16: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,DATE,TITLE,TOTAL_JOBS
0,2025-04-21,Data Analyst,3
1,2025-04-22,Data Analyst,13
2,2025-04-23,Data Analyst,11
3,2025-04-24,Data Analyst,10
4,2025-04-25,Data Analyst,3


In [ ]:
import streamlit as st
import plotly_express as px

def viz_daily_job_data(df_daily_jobs):
    fig = px.line(df_daily_jobs, x='DATE', y='TOTAL_JOBS', color='TITLE',
                  title='Data Job Daily Trend')
    st.plotly_chart(fig)

viz_daily_job_data(df_daily_jobs)

In [ ]:
import sys
print(sys.executable)
import nbformat
print(nbformat.__version__)

In [50]:
import seaborn as sns
import matplotlib.pyplot as plt

def viz_daily_job_data(df_daily_jobs):
    plt.figure(figsize=(10,6))
    sns.lineplot(data=df_daily_jobs, x='DATE', y='TOTAL_JOBS', hue='TITLE')
    plt.title('Data Job Daily Trend')
    plt.xlabel('Date')
    plt.ylabel('Total Jobs')
    plt.legend(title='Job Title')
    plt.tight_layout()
    plt.show()

viz_daily_job_data(df_daily_jobs)

## Section  - Dashboard Title


In [22]:
#query the min and max dates for dashboard subheading

def job_dates(conn):
    city_query_base = """
        SELECT DISTINCT JOB_CATEGORY AS Job_Role,
        ID, TITLE, EMPLOYMENT_TYPE, SENIORITY, CITY, STATE, ORGANIZATION,
        URL AS LinkedIn_Post_Link, JOB_DATE AS Job_Posted_Date,
        REMOTE_DERIVED AS IS_REMOTE,
        LINKEDIN_ORG_URL,
        LINKEDIN_ORG_INDUSTRY,
        LINKEDIN_ORG_RECRUITMENT_AGENCY_DERIVED AS Job_By_Agency,
        DIRECTAPPLY
        FROM INT_LINKEDIN_DATA
        ORDER BY JOB_DATE, JOB_CATEGORY
                            """

    df = pd.read_sql(city_query_base, conn)
    conn.close()
    return df

df = job_dates(conn)
df

/tmp/ipykernel_10361/1405250934.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(city_query_base, conn)


,JOB_ROLE,ID,TITLE,EMPLOYMENT_TYPE,SENIORITY,CITY,STATE,ORGANIZATION,LINKEDIN_POST_LINK,JOB_POSTED_DATE,IS_REMOTE,LINKEDIN_ORG_URL,LINKEDIN_ORG_INDUSTRY,JOB_BY_AGENCY,DIRECTAPPLY
0,Data Analyst,1583501963,Health & Safety Data Analyst,FULL_TIME,Entry Level,Melbourne,VIC,Suburban Connect,https://au.linkedin.com/jobs/view/health-safet...,2025-04-21,False,None,None,None,False
1,Data Analyst,1583892935,Data Analyst,FULL_TIME,Entry Level,Sydney,NSW,Data Insight Labs Pvt ltd,https://au.linkedin.com/jobs/view/data-analyst...,2025-04-21,False,None,None,None,True
2,Data Analyst,1585161617,Functional Data Analyst,CONTRACTOR,Not Applicable,Brisbane,QLD,Peoplebank,https://au.linkedin.com/jobs/view/functional-d...,2025-04-21,False,http://www.peoplebank.com.au,Staffing and Recruiting,True,False
3,Data Scientist,1583444447,Data Scientist,OTHER,Entry Level,Perth,WA,AI Connect Advisory,https://au.linkedin.com/jobs/view/data-scienti...,2025-04-21,False,https://www.aiconnectadvisory.com,"Technology, Information and Internet",False,False
4,Data Scientist,1583995993,Data Scientist,FULL_TIME,Senior,Brisbane,QLD,Yum! Brands,https://au.linkedin.com/jobs/view/data-scienti...,2025-04-21,False,http://www.yum.com,Restaurants,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,Data Engineer,1638624041,Data Engineer,CONTRACTOR,Senior,Melbourne,VIC,Avance Consulting,https://au.linkedin.com/jobs/view/data-enginee...,2025-05-09,False,https://www.avanceservices.com,Staffing and Recruiting,True,True
387,Data Engineer,1638053647,Neo4j / AWS Neptune - Graph Data Engineer / T...,FULL_TIME,Senior,Sydney,NSW,Private Advertiser,https://au.linkedin.com/jobs/view/neo4j-aws-ne...,2025-05-09,False,http://www.seek.com.au/job/29946059?pos=1&type...,Mining,False,True
388,Data Engineer,1638778785,Data Engineer - Amazon FinTech,FULL_TIME,Not Applicable,Sydney,NSW,Amazon,https://au.linkedin.com/jobs/view/data-enginee...,2025-05-09,False,https://www.aboutamazon.com/,Software Development,False,False
389,Data Engineer,1638162209,Senior Data Engineer,FULL_TIME,Senior,Sydney,NSW,Tranzformd,https://au.linkedin.com/jobs/view/senior-data-...,2025-05-09,False,https://tranzformd.com.au/,IT Services and IT Consulting,False,False


In [ ]:
st.title("Australia Data Job Trend Dashboard")
# st.subheader(f"data date range {df_dates['MIN_DATE']} - {df_dates['MAX_DATE']}")

In [5]:
import pandas as pd
from datetime import datetime, timedelta


def viz_wow_trend():
    today = datetime.now()
    current_week = today - timedelta(days=today.weekday())
    current_week = pd.to_datetime(current_week).date()
    print(current_week)


current_week = viz_wow_trend()
current_week

2025-05-05
